In [4]:
%ls

kenlm/                                地址数据/
lm_corpus/                            整理保险语言模型语料.ipynb
sighan bakeroff/                      整理测试语料.ipynb
保险语料/


In [5]:
import xml.etree.ElementTree as ET

In [6]:
from tqdm import tqdm
import os
import re

# 格式一

In [7]:
flist1 = os.listdir("sighan bakeroff/training/格式1")

In [8]:
merged_dict1 = {}
for f in flist1:
    tree = ET.parse("sighan bakeroff/training/格式1/"+f)
    root = tree.getroot()
    for essay in tqdm(root, desc=f):
        # TEXT[Passage], MISTAKE[WRONG, CORRECTION]

        passages = essay.findall("TEXT/PASSAGE")
        mistakes = essay.findall("MISTAKE")

        for passage in passages:
            id = passage.attrib['id']
            if id in merged_dict1:
                print("Warning: id {} already exists".format(id))
            merged_dict1[id] = {"original": passage.text, "mistakes": []}

        for mistake in mistakes:
            mistake_id = mistake.attrib['id']
            wrong = mistake.find('WRONG').text
            correct = mistake.find('CORRECTION').text
            merged_dict1[mistake_id]['mistakes'].append((wrong, correct))

C1_training.txt: 100%|██████████| 114/114 [00:00<00:00, 78669.08it/s]


# 格式二

In [11]:
flist2 = os.listdir("sighan bakeroff/training/格式2")
print(flist2)

['Bakeoff2013_SampleSet_WithError_00001-00350.txt', 'Bakeoff2013_SampleSet_WithoutError_10001-10350.txt']


In [12]:
merged_dict2 = {}
for f in flist2:
    tree = ET.parse("sighan bakeroff/training/格式2/"+f, )
    root = tree.getroot()
    for doc in tqdm(root, desc=f):
        doc_id = doc.attrib['Nid']
        passage = doc.find("P")
        merged_dict2[doc_id] = {'original': passage.text, "mistakes": []}
        mistakes = doc.findall("MISTAKE")

        for mistake in mistakes:
            wrong = mistake.find('WRONG').text
            correct = mistake.find('CORRECTION').text
            merged_dict2[doc_id]['mistakes'].append((wrong, correct))

Bakeoff2013_SampleSet_WithError_00001-00350.txt: 100%|██████████| 350/350 [00:00<00:00, 521679.60it/s]
Bakeoff2013_SampleSet_WithoutError_10001-10350.txt: 100%|██████████| 350/350 [00:00<00:00, 611669.33it/s]


# Test

In [13]:
flist3 = os.listdir("sighan bakeroff/test")
print(flist3)

['DryRun_SubTask2.txt', 'FinalTest_SubTask2Truth.txt', 'DryRun_SubTask1.txt', 'DryRun_SubTask1Truth.txt', 'SIGHAN15_CSC_DryTruth.txt', 'CLP14_CSC_DryRunTruth.txt', 'CLP14_CSC_TestTruth.txt', 'FinalTest_SubTask2.txt', 'DryRun_SubTask2Truth.txt', 'SIGHAN15_CSC_Dry.txt', 'CLP14_CSC_DryRun.txt', 'CLP14_CSC_Test.txt', 'SIGHAN15_CSC_Test.txt', 'SIGHAN15_CSC_TestTruth.txt']


In [14]:
merged_dict3 = {}
for f in flist3:
    if 'Truth' not in f:
        continue
    f2 = f.replace("Truth", "")
    with open("sighan bakeroff/test/"+f) as truth_f:
        with open("sighan bakeroff/test/"+f2) as input_f:
            for inp, tru in tqdm(zip(input_f.readlines(), truth_f.readlines()), desc=f2):
#                 print(inp, tru)
                if inp == "":
                    continue
                inp_id = re.findall("^\\(.(?:ID|id)=[A-Z\d-]*\\)", inp)
                inp_text = inp.replace(inp_id[0], "").strip()

                idd = inp_id[0].split("=")[1][:-1]
                if idd in merged_dict3:
                    print("Warning: {}/{} already exists".format(f2, idd))
                merged_dict3[idd] = {"original": inp_text, "mistakes":[]}
                
                tru_split = tru.split(",")
                tru_id = tru_split[0].strip()
                assert tru_id == idd
                corrections = [(int(i), j.strip()) for i, j in zip(tru_split[1:][::2], tru_split[1:][1::2])]
                merged_dict3[idd]['mistakes'].extend(corrections)

FinalTest_SubTask2.txt: 1000it [00:00, 139628.62it/s]
DryRun_SubTask1.txt: 50it [00:00, 89660.20it/s]
SIGHAN15_CSC_Dry.txt: 39it [00:00, 113046.20it/s]
CLP14_CSC_DryRun.txt: 20it [00:00, 73519.79it/s]
CLP14_CSC_Test.txt: 1062it [00:00, 153577.12it/s]
DryRun_SubTask2.txt: 50it [00:00, 87417.76it/s]
SIGHAN15_CSC_Test.txt: 1100it [00:00, 139101.98it/s]


# Merge and correct

In [15]:
len(merged_dict1) + len(merged_dict2) + len(merged_dict3) 

9797

In [16]:
list_with_error = []
list_no_error = []

for item in tqdm(merged_dict1):
    text = merged_dict1[item]['original']
    correction = text
    for mistake in merged_dict1[item]['mistakes']:
        correction = correction.replace(mistake[0], mistake[1])
    if correction != text:
        list_with_error.append((text, correction))
    else:
        list_no_error.append((text, correction))
        
for item in tqdm(merged_dict2):
    text = merged_dict2[item]['original']
    correction = text
    for mistake in merged_dict2[item]['mistakes']:
        correction = correction.replace(mistake[0], mistake[1])
    if correction != text:
        list_with_error.append((text, correction))
    else:
        list_no_error.append((text, correction))
        
for item in tqdm(merged_dict3):
    text = merged_dict3[item]['original']
    correction = text
    for mistake in merged_dict3[item]['mistakes']:
        mistake_idx = mistake[0]
        replacement = mistake[1]
        
        correction = correction[:mistake_idx-1]+replacement+correction[mistake_idx:]
    if correction != text:
        list_with_error.append((text, correction))
    else:
        list_no_error.append((text, correction))

100%|██████████| 3321/3321 [00:00<00:00, 482700.34it/s]


In [17]:
len(list_with_error)

7750

In [18]:
len(list_no_error)

2047

# Write to csv

In [19]:
import csv
from csv import QUOTE_MINIMAL

In [23]:
import random
random.seed(1234)

In [25]:
test_data = random.sample(list_with_error, 1500) + random.sample(list_no_error, 500)

In [29]:
train_data = [l1 for l1 in list_with_error+list_no_error if l1 not in test_data]

In [30]:
len(train_data)

7791

In [31]:
with open('sighan bakeroff/train.csv', 'w') as f:
    for item in train_data:
        csv_writer = csv.writer(f, quoting=QUOTE_MINIMAL)
        csv_writer.writerow(item)

In [32]:
with open('sighan bakeroff/test.csv', 'w') as f:
    for item in test_data:
        csv_writer = csv.writer(f, quoting=QUOTE_MINIMAL)
        csv_writer.writerow(item)